In [1]:
import os
import glob
import sqlite3
import sys
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from agents import Agent, Runner, function_tool
from agents.tracing import trace
from datetime import datetime
import sqlite3
from typing import Dict, Any, List
import gradio as gr


c:\Users\Ziyad\Desktop\newww\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = "gpt-5.2-2025-12-11"
db_name = "vector_db"
load_dotenv(override=True)
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


OpenAI API Key exists and begins sk-proj-


In [3]:
from typing import Any, Dict, Optional
import sqlite3

@function_tool
def property_search(
    price: Optional[str] = None,
    rooms: Optional[int] = None,
    location: Optional[str] = None
) -> Dict[str, Any]:
    """
    Optional filters (user can pass any one, or multiple):
    - price: numeric string containing digits only (e.g., "2000000"). No abbreviations, symbols, commas, or currency labels.
    - rooms: number of bedrooms as an integer
    - location: valid UAE area name (normalize abbreviations/spelling; e.g., JVC -> Jumeirah Village Circle)
    SQL query parameters must be in English.
    """

    # --- normalize price (only if provided) ---
    max_price: Optional[int] = None
    if price is not None and str(price).strip():
        p = str(price).lower().replace(",", "").strip()
        # If you truly require digits-only, remove this "m" handling.
        if "m" in p:
            max_price = int(float(p.replace("m", "").strip()) * 1_000_000)
        else:
            max_price = int(p)

    # --- build dynamic SQL ---
    where_clauses: list[str] = []
    params: list[Any] = []

    if max_price is not None:
        where_clauses.append("price_aed <= ?")
        params.append(max_price)

    if rooms is not None:
        where_clauses.append("bedrooms >= ?")
        params.append(int(rooms))

    if location is not None and str(location).strip():
        loc = str(location).strip()
        where_clauses.append("LOWER(area) LIKE LOWER(?)")
        params.append(f"%{loc}%")

    if not where_clauses:
        return {"ok": False, "error": "Provide at least one filter: price, rooms, or location."}

    where_sql = " AND ".join(where_clauses)

    # Ordering: if price is provided, rank by closeness to that price; otherwise cheapest first.
    if max_price is not None:
        order_sql = "ORDER BY ABS(price_aed - ?) ASC"
        params_for_query = params + [max_price]  # extra param for ORDER BY expression
    else:
        order_sql = "ORDER BY price_aed ASC"
        params_for_query = params

    query = f"""
    SELECT listing_id, title, price_aed, bedrooms, area, handover, image_url, description
    FROM prop_table
    WHERE {where_sql}
    {order_sql}
    LIMIT 7
    """

    # --- execute ---
    conn = sqlite3.connect("properties.db")
    try:
        cursor = conn.cursor()
        cursor.execute(query, params_for_query)
        rows = cursor.fetchall()
    finally:
        conn.close()

    results = []
    for (listing_id, title, price_aed, bedrooms, area, handover, image_url, description) in rows:
        results.append({
            "listing_id": listing_id,
            "title": title,
            "price_aed": price_aed,
            "bedrooms": bedrooms,
            "area": area,
            "handover": handover,
            "image_url": image_url,
            "description": description,
        })

    if not results:
        return {"ok": True, "message": "No properties found matching the criteria.", "results": []}

    return {"ok": True, "results": results}



In [4]:
# # for querying db

# conn = sqlite3.connect("properties.db")
# cursor = conn.cursor()
#   # --- SQL query (hard filters) ---
# query = """
#     SELECT *
#     FROM prop_table
#     where price_aed <= '4000000'
#     and bedrooms = 4
#     LIMIT 3
#     """

# cursor.execute(query)
# rows = cursor.fetchall()
# print(rows)
# conn.close()

In [5]:
instructionss =  """
You are a knowledgeable, friendly assistant representing real estate broker who speciallises in off-plan properties in the UAE.
You have a nuanced understanding of the nuances of the UAE market and the rules for the golden visa.
You are chatting with foreign investors who are interested in buying off-plan properties in the UAE.
Use property_search tool to find suitable properties and recommend them to the investor. Explain why it suits them. Be concise.
If the property_search tool returns no results, ask the user to refine their criteria (e.g., provide a more specific location or adjust budget) without mentioning that the tool failed.
Recommend a maximum of 3 properties at any time. Never recommend more than 3 in one response. If more than 3 are returned by the tool, choose the best 3 based on the property description.
"""

tools = [property_search]

agent = Agent(
    name="Broker Agent",
    instructions=instructionss,
    tools=tools,
    model=MODEL
)

trace_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")




In [6]:
def _extract_properties_from_result(run_result) -> List[Dict[str, Any]]:
    properties: List[Dict[str, Any]] = []

    for item in getattr(run_result, "new_items", []) or []:
        output = getattr(item, "output", None)
        if isinstance(output, dict) and isinstance(output.get("results"), list):
            properties = output["results"][:3]

    return properties[:3]


def _safe_price(value: Any) -> str:
    try:
        return f"AED {int(value):,}"
    except (TypeError, ValueError):
        return "AED N/A"


def _format_property_text(prop: Dict[str, Any]) -> str:
    description = str(prop.get("description") or "No description available.").strip()

    return (
        f"ID: {prop.get('listing_id', 'N/A')}\n"
        f"Title: {prop.get('title', 'Untitled property')}\n"
        f"Price: {_safe_price(prop.get('price_aed'))}\n"
        f"Bedrooms: {prop.get('bedrooms', 'N/A')}\n"
        f"Area: {prop.get('area', 'N/A')}\n"
        f"Handover: {prop.get('handover', 'N/A')}\n\n"
        f"Description:\n{description}"
    )


def _build_property_outputs(properties: List[Dict[str, Any]]):
    images = [None, None, None]
    texts = ["", "", ""]

    for idx, prop in enumerate(properties[:3]):
        image_url = prop.get("image_url")
        images[idx] = str(image_url) if image_url else None
        texts[idx] = _format_property_text(prop)

    return images, texts


def normalize_part_type(role: str, part: dict) -> dict:
    text_value = str(part.get("text", ""))
    if role == "user":
        return {"type": "input_text", "text": text_value}
    return {"type": "output_text", "text": text_value}


def to_responses_input(history, new_user_message: str):
    items = []

    for item in history:
        if isinstance(item, (list, tuple)) and len(item) == 2:
            user_msg, assistant_msg = item
            if user_msg:
                items.append({"role": "user", "content": [{"type": "input_text", "text": str(user_msg)}]})
            if assistant_msg:
                items.append({"role": "assistant", "content": [{"type": "output_text", "text": str(assistant_msg)}]})
            continue

        if isinstance(item, dict) and "role" in item:
            role = item["role"]
            content = item.get("content", "")

            if isinstance(content, list):
                fixed_parts = []
                for part in content:
                    if isinstance(part, dict) and part.get("type") in ("text", "input_text", "output_text"):
                        fixed_parts.append(normalize_part_type(role, part))
                if fixed_parts:
                    items.append({"role": role, "content": fixed_parts})
            elif isinstance(content, str):
                part_type = "input_text" if role == "user" else "output_text"
                items.append({"role": role, "content": [{"type": part_type, "text": content}]})

    items.append({"role": "user", "content": [{"type": "input_text", "text": new_user_message}]})
    return items


async def agent_chat(message, history):
    inp = to_responses_input(history, message)

    with trace(trace_id):
        result = await Runner.run(agent, input=inp)

    properties = _extract_properties_from_result(result)
    images, texts = _build_property_outputs(properties)

    return (
        str(result.final_output),
        images[0], texts[0],
        images[1], texts[1],
        images[2], texts[2],
    )


with gr.Blocks() as demo:
    prop1_img = gr.Image(label="Property 1", render=False)
    prop1_desc = gr.Textbox(label="Description 1", lines=10, interactive=False, render=False)

    prop2_img = gr.Image(label="Property 2", render=False)
    prop2_desc = gr.Textbox(label="Description 2", lines=10, interactive=False, render=False)

    prop3_img = gr.Image(label="Property 3", render=False)
    prop3_desc = gr.Textbox(label="Description 3", lines=10, interactive=False, render=False)

    gr.ChatInterface(
        fn=agent_chat,
        title="Broker Agent",
        additional_outputs=[prop1_img, prop1_desc, prop2_img, prop2_desc, prop3_img, prop3_desc]
    )

    with gr.Accordion("Recommended properties", open=True):
        with gr.Row():
            with gr.Column():
                prop1_img.render()
                prop1_desc.render()
            with gr.Column():
                prop2_img.render()
                prop2_desc.render()
            with gr.Column():
                prop3_img.render()
                prop3_desc.render()


demo.launch()



* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
